# Testing accuracy of Stock Price Predictions made using XGBOOST

In [25]:
!pip install scikit-learn
!pip install pandas
!pip install plotly
import sys
!{sys.executable} -m pip install xgboost

You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.



  Using cached xgboost-1.4.2-py3-none-win_amd64.whl (97.8 MB)


You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [29]:
from IPython.core.debugger import set_trace

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import time
import yfinance as yf
plt.style.use(style="seaborn")
%matplotlib inline

In [36]:
df = pd.read_csv("data/MSFT-1Y-Hourly.csv")
df.dtypes

date         object
open        float64
high        float64
low         float64
close       float64
volume        int64
average     float64
barCount      int64
dtype: object

In [37]:
df.head(5)

,date,open,high,low,close,volume,average,barCount
0,2019-08-07 14:30:00,133.80,133.83,131.82,132.89,35647,132.701,17523
1,2019-08-07 15:00:00,132.87,135.20,132.64,134.75,48757,134.043,26974
2,2019-08-07 16:00:00,134.74,134.92,133.52,133.88,28977,134.147,17853
3,2019-08-07 17:00:00,133.89,134.06,133.07,133.90,21670,133.618,13497
4,2019-08-07 18:00:00,133.89,135.24,133.83,134.83,22648,134.653,12602


In [39]:
df = df[["close"]].copy()

In [40]:
df.head(5)

,close
0,132.89
1,134.75
2,133.88
3,133.90
4,134.83


## Transform this to a supervised learning problem.

In [42]:
df["target"] = df['close'].shift(-1)


In [43]:
df.dropna(inplace=True)

In [44]:
df.head(5)

,close,target
0,132.89,134.75
1,134.75,133.88
2,133.88,133.90
3,133.90,134.83
4,134.83,135.48


### Train Test data Split

In [45]:
def train_test_split(data, perc):
    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

In [46]:
train, test = train_test_split(df, 0.2)
print(len(train))
print(len(test))

1401
351


Take the train set and test input row as input, fit a model, and make a prediction using XGBRegressor of scikit-learn.

In [47]:
X = train[:, :-1]
y = train[:, -1]

In [48]:
y

array([134.75, 133.88, 133.9 , ..., 183.53, 183.51, 184.08])

In [49]:
model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [50]:
test[0]

array([184.08, 183.74])

In [51]:
val = np.array(test[0, 0]).reshape(1, -1)

pred = model.predict(val)
print(pred[0])

184.6561


### Predict one sample at a time

In [52]:
def xgb_predict(train, val):
    train = np.array(train)
    X, y = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    model.fit(X, y)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

In [53]:
xgb_predict(train, test[0, 0])

184.6561

### Walk-forward validation

 Making a one step forward prediction, then adding the real observation from the test set to the train set, refit the model, then predict the next step in the test dataset.

In [55]:
from sklearn.metrics import mean_squared_error


def validate(data, perc):
    predictions = []

    train, test = train_test_split(data, perc)

    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]

        pred = xgb_predict(history, test_X[0])
        predictions.append(pred)

        history.append(test[i])

    error = mean_squared_error(test[:, -1], predictions, squared=False)

    return error, test[:, -1], predictions

In [ ]:
%%time
rmse, y, pred = validate(df, 0.2)

print(rmse)

### So our predicted model has low RMSE compared to the target values. But still it is not the most accurate model to predict stock price trend. But for long term investements, one may trust this predictions.